## 🔹 ETAPA 1 - EXTRAÇÃO

✅ Escolher uma API

✅ Fazer a extração de pelo menos 3 tabelas 


In [1]:
import requests
import pandas as pd
from numpy import dtype as np
from datetime import datetime
import sqlite3

# URLs da API
url_produto = 'https://fakestoreapi.com/products'
url_usuarios = 'https://fakestoreapi.com/users'
url_carrinhos = 'https://fakestoreapi.com/carts'

# Requisição para os produtos
res_produtos = requests.get(url_produto)
if res_produtos.status_code == 200:
    df_produtos = pd.DataFrame(res_produtos.json())
    print("Tabela de Produtos")
    print(df_produtos.head(5))
else:
    print(f"Erro ao acessar produtos: {res_produtos.status_code}")

# Requisição para os usuários
res_usuarios = requests.get(url_usuarios)
if res_usuarios.status_code == 200:
    df_usuarios = pd.DataFrame(res_usuarios.json())
    print("Tabela de Usuários")
    print(df_usuarios.head(5))
else:
    print(f"Erro ao acessar usuários: {res_usuarios.status_code}")

# Requisição para os carrinhos
res_carrinhos = requests.get(url_carrinhos)
if res_carrinhos.status_code == 200:
    df_carrinhos = pd.DataFrame(res_carrinhos.json())
    print("Tabela de Carrinhos")
    print(df_carrinhos.head(5))
else:
    print(f"Erro ao acessar carrinhos: {res_carrinhos.status_code}")



Tabela de Produtos
   id                                              title   price  \
0   1  Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...  109.95   
1   2             Mens Casual Premium Slim Fit T-Shirts    22.30   
2   3                                 Mens Cotton Jacket   55.99   
3   4                               Mens Casual Slim Fit   15.99   
4   5  John Hardy Women's Legends Naga Gold & Silver ...  695.00   

                                         description        category  \
0  Your perfect pack for everyday use and walks i...  men's clothing   
1  Slim-fitting style, contrast raglan long sleev...  men's clothing   
2  great outerwear jackets for Spring/Autumn/Wint...  men's clothing   
3  The color could be slightly different between ...  men's clothing   
4  From our Legends Collection, the Naga was insp...        jewelery   

                                               image  \
0  https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...   
1  https://fakestoreapi.com

## 🔹 ETAPA 2 - TRATAMENTO DE DADOS

✅ Realizar os tratamentos devidos das 3 tabelas = 'Produtos' - 'Usuarios' - 'Carrinho'

In [ ]:
df_produtos.rename(columns={
    'id': 'id_produto', 
    'title': 'nome_produto', 
    'price': 'preco', 
    'description': 'descricao', 
    'category': 'categoria', 
    'image': 'imagem',
    'rating': 'avaliacao'
    }, inplace=True)


if 'avaliacao' in df_produtos.columns:

    df_produtos['avaliar'] = df_produtos['avaliacao'].apply(
        lambda x: x['rate'] if isinstance(x, dict) and 'rate' in x else None
    )
    df_produtos['contar'] = df_produtos['avaliacao'].apply(
        lambda x: x['count'] if isinstance(x, dict) and 'count' in x else None
    )
else:
    print("A coluna 'avaliacao' não está presente no DataFrame.")

# Transformar para float
df_produtos['preco'] = df_produtos['preco'].astype(float)
df_produtos['id_produto'] = df_produtos['id_produto'].astype(int)

# Remover colunas desnecessárias
df_produtos.drop(columns=['imagem'], inplace=True)
df_produtos.drop(columns=['avaliacao'], inplace=True)

# posicionar as colunas
df_produtos = df_produtos[['id_produto', 'nome_produto', 'preco', 'categoria', 'avaliar', 'contar', 'descricao']]

df_produtos.head(5)

,id_produto,nome_produto,preco,categoria,avaliar,contar,descricao
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,men's clothing,3.9,120,Your perfect pack for everyday use and walks i...
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,men's clothing,4.1,259,"Slim-fitting style, contrast raglan long sleev..."
2,3,Mens Cotton Jacket,55.99,men's clothing,4.7,500,great outerwear jackets for Spring/Autumn/Wint...
3,4,Mens Casual Slim Fit,15.99,men's clothing,2.1,430,The color could be slightly different between ...
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,jewelery,4.6,400,"From our Legends Collection, the Naga was insp..."


In [3]:
print(df_produtos.dtypes)

id_produto        int64
nome_produto     object
preco           float64
categoria        object
avaliar         float64
contar            int64
descricao        object
dtype: object


In [4]:
# Renomear as colunas para nomes mais legíveis
df_usuarios.rename(columns={
    'name': 'nome',
    'address': 'endereco',
    'id': 'id_usuario',
    'email': 'e_mail',
    'username': 'usuario',
    'password': 'senha',
    'phone': 'telefone'
}, inplace=True)


# Criar coluna 'nome_completo' se a coluna 'nome' existir
if 'nome' in df_usuarios.columns:
    df_usuarios['nome_completo'] = df_usuarios['nome'].apply(
        lambda x: f"{x['firstname']} {x['lastname']}" if isinstance(x, dict) else ''
    )

# Extrair informações de endereço
if 'endereco' in df_usuarios.columns:
    
    df_usuarios['rua'] = df_usuarios['endereco'].apply(
        lambda x: x['street'] if isinstance(x, dict) and 'street' in x else ''
    )
    df_usuarios['cidade'] = df_usuarios['endereco'].apply(
        lambda x: x['city'] if isinstance(x, dict) and 'city' in x else ''
    )
    df_usuarios['numero'] = df_usuarios['endereco'].apply(
        lambda x: x['number'] if isinstance(x, dict) and 'number' in x else ''
    )
    df_usuarios['cep'] = df_usuarios['endereco'].apply(
        lambda x: x['zipcode'] if isinstance(x, dict) and 'zipcode' in x else ''
    )

    # Extrair latitude e longitude
    df_usuarios['latitude'] = df_usuarios['endereco'].apply(
        lambda x: x['geolocation']['lat'] if isinstance(x, dict) and 'geolocation' in x else ''
    )
    df_usuarios['longitude'] = df_usuarios['endereco'].apply(
        lambda x: x['geolocation']['long'] if isinstance(x, dict) and 'geolocation' in x else ''
    )

# Transformar para float
df_usuarios['latitude'] = df_usuarios['latitude'].astype(float)
df_usuarios['longitude'] = df_usuarios['longitude'].astype(float)

# Remover colunas desnecessárias
df_usuarios.drop(columns=['endereco'], inplace=True)
df_usuarios.drop(columns=['nome'], inplace=True)
df_usuarios.drop(columns=['__v'], inplace=True)

# posicionar as colunas
df_usuarios = df_usuarios[['id_usuario', 'nome_completo', 'usuario', 'senha', 'e_mail', 'telefone', 'rua', 'numero', 'cidade', 'cep', 'latitude', 'longitude']]


df_usuarios.head(5)


,id_usuario,nome_completo,usuario,senha,e_mail,telefone,rua,numero,cidade,cep,latitude,longitude
0,1,john doe,johnd,m38rmF$,john@gmail.com,1-570-236-7033,new road,7682,kilcoole,12926-3874,-37.3159,81.1496
1,2,david morrison,mor_2314,83r5^_,morrison@gmail.com,1-570-236-7033,Lovers Ln,7267,kilcoole,12926-3874,-37.3159,81.1496
2,3,kevin ryan,kevinryan,kev02937@,kevin@gmail.com,1-567-094-1345,Frances Ct,86,Cullman,29567-1452,40.3467,-30.1310
3,4,don romer,donero,ewedon,don@gmail.com,1-765-789-6734,Hunters Creek Dr,6454,San Antonio,98234-1734,50.3467,-20.1310
4,5,derek powell,derek,jklg*_56,derek@gmail.com,1-956-001-1945,adams St,245,san Antonio,80796-1234,40.3467,-40.1310


In [5]:

print(df_usuarios.dtypes)

id_usuario         int64
nome_completo     object
usuario           object
senha             object
e_mail            object
telefone          object
rua               object
numero             int64
cidade            object
cep               object
latitude         float64
longitude        float64
dtype: object


In [6]:
# Renomear colunas
df_carrinhos.rename(columns={
    'id': 'id_carrinho',
    'userId': 'id_usuario',
    'products': 'produtos',
    'date': 'data_criacao',
}, inplace=True)

# Converter data
df_carrinhos['data_criacao'] = pd.to_datetime(df_carrinhos['data_criacao'])


df_carrinhos.drop(columns=['__v'], inplace=True)

# Garante que todos os valores da coluna sejam listas
df_carrinhos['produtos'] = df_carrinhos['produtos'].apply(lambda x: x if isinstance(x, list) else [])

# Explode os produtos (transforma cada produto em uma linha separada)
df_carrinhos = df_carrinhos.explode('produtos').reset_index(drop=True)

# Extrai campos de cada produto
df_carrinhos['id_produto'] = df_carrinhos['produtos'].apply(
    lambda x: x['productId'] if isinstance(x, dict) and 'productId' in x else None
)

df_carrinhos['quantidade'] = df_carrinhos['produtos'].apply(
    lambda x: x['quantity'] if isinstance(x, dict) and 'quantity' in x else None
)

df_carrinhos = df_carrinhos[['id_carrinho', 'id_usuario', 'id_produto', 'quantidade', 'data_criacao']]


df_carrinhos.head()


,id_carrinho,id_usuario,id_produto,quantidade,data_criacao
0,1,1,1,4,2020-03-02 00:00:00+00:00
1,1,1,2,1,2020-03-02 00:00:00+00:00
2,1,1,3,6,2020-03-02 00:00:00+00:00
3,2,1,2,4,2020-01-02 00:00:00+00:00
4,2,1,1,10,2020-01-02 00:00:00+00:00


In [7]:
print(df_carrinhos.dtypes)

id_carrinho                   int64
id_usuario                    int64
id_produto                    int64
quantidade                    int64
data_criacao    datetime64[ns, UTC]
dtype: object


## 🔹 ETAPA 3 - CARREGAMENTO 

✅ Carregar as informações para o SQLITE3

In [8]:
conn = sqlite3.connect('loja.db')
cursor = conn.cursor()


# Criar tabela 'produtos'
cursor.execute('''
    CREATE TABLE IF NOT EXISTS produtos (
        id_produto INTEGER PRIMARY KEY,
        nome_produto TEXT,
        preco float,
        categoria TEXT,
        avaliar float,
        contar INTEGER,
        descricao TEXT
    )
''')

# Criar tabela 'usuarios'
cursor.execute('''
    CREATE TABLE IF NOT EXISTS usuarios (
        id_usuario INTEGER PRIMARY KEY,
        nome_completo TEXT,
        usuario TEXT,
        senha TEXT,
        e_mail TEXT,
        telefone TEXT,
        rua TEXT,
        numero TEXT,
        cidade TEXT,
        cep TEXT,
        latitude float,
        longitude float
    )
''')

# Criar tabela 'carrinhos'
cursor.execute('''
    CREATE TABLE IF NOT EXISTS carrinhos (
        id_carrinho INTEGER PRIMARY KEY,
        id_usuario INTEGER,
        id_produto INTEGER,
        quantidade INTEGER,
        data_criacao DATE,
        FOREIGN KEY (id_usuario) REFERENCES usuarios (id_usuario),
        FOREIGN KEY (id_produto) REFERENCES produtos (id_produto)
    )
''')

df_produtos.to_sql('produtos', conn, if_exists='append', index=False)
df_usuarios.to_sql('usuarios', conn, if_exists='append', index=False)
df_carrinhos.to_sql('carrinhos', conn, if_exists='replace', index=False)

conn.commit()
conn.close()

print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


## 🔹 ETAPA 4 - VALIDAÇÃO

✅ Validar as tabelas disponibilizadas

In [9]:
import sqlite3

conn = sqlite3.connect('loja.db')
cursor = conn.cursor()


cursor.execute("SELECT COUNT(*) FROM produtos")
print('produtos:', {cursor.fetchone()[0]})

produtos: {20}


In [10]:
cursor.execute("SELECT COUNT(*) FROM usuarios")
print('usuarios:', {cursor.fetchone()[0]})

usuarios: {10}


In [11]:
cursor.execute("SELECT COUNT(*) FROM carrinhos")
print('carrinhos:', {cursor.fetchone()[0]})

conn.close()

carrinhos: {14}


## 🔹 ETAPA 5 - SISTEMA DE ALERTA

✅ Alerta, caso o usuario faça um registro sem o e_mail cadastrado

In [12]:
from plyer import notification

def notificar_usuario(e_mail, df_usuarios):
    # Verifica se o e-mail foi fornecido
    if not e_mail or not isinstance(e_mail, str):
        print("Erro: E-mail inválido ou vazio.")
        return

    # Verifica se a coluna 'e_mail' existe no DataFrame
    if 'e_mail' not in df_usuarios.columns:
        print("Erro: A coluna 'e_mail' não existe no DataFrame.")
        return

    # Verifica se o e-mail está cadastrado
    if e_mail in df_usuarios['e_mail'].values:
        print(f"E-mail '{e_mail}' encontrado na base de dados.")
    else:
        print(f"E-mail '{e_mail}' NÃO encontrado na base de dados. Notificando...")
        notification.notify(
            title='Alerta de Notificação',
            message=f'Atenção! O e-mail "{e_mail}" não está cadastrado!',
            app_name='Loja Fake Store',
            timeout=10
        )




In [14]:
import pandas as pd

# Exemplo de uso, voce pode alterar o e-mail para testar
df_usuarios = pd.DataFrame({'e_mail': ['don@gmail.com']}
)

notificar_usuario('gab@gmail.com', df_usuarios)

E-mail 'gab@gmail.com' NÃO encontrado na base de dados. Notificando...
